In [1]:
# 모듈 import
from mmcv import Config
from mmdet.datasets import build_dataset
from mmdet.models import build_detector
from mmdet.apis import train_detector
from mmdet.datasets import (build_dataloader, build_dataset,
                            replace_ImageToTensor)
from mmdet.utils import get_device

/opt/conda/envs/detection/lib/python3.7/site-packages/mmcv/__init__.py:21: UserWarning: On January 1, 2023, MMCV will release v2.0.0, in which it will remove components related to the training process and add a data transformation module. In addition, it will rename the package names mmcv to mmcv-lite and mmcv-full to mmcv. See https://github.com/open-mmlab/mmcv/blob/master/docs/en/compatibility.md for more details.
  'On January 1, 2023, MMCV will release v2.0.0, in which it will remove '


In [3]:
classes = ("General trash", "Paper", "Paper pack", "Metal", "Glass", 
           "Plastic", "Styrofoam", "Plastic bag", "Battery", "Clothing")

# config file 들고오기
#cfg = Config.fromfile('./configs/faster_rcnn/faster_rcnn_r50_fpn_1x_coco.py')
#cfg=Config.fromfile('./configs/yolof/yolof_r50_c5_8x8_1x_coco.py')
cfg=Config.fromfile('./configs/yolof/yolof_r50_c5_8x8_iter-1x_coco.py')
root='../../dataset/'

# dataset config 수정
cfg.data.train.classes = classes
cfg.data.train.img_prefix = root
#stratified kfold
cfg.data.train.ann_file=root+'stratified_v2/train_fold_1.json'
#cfg.data.train.ann_file = root + 'train.json' # train json 정보
cfg.data.train.pipeline[2]['img_scale'] = (512,512) # Resize



cfg.data.test.classes = classes
cfg.data.test.img_prefix = root
cfg.data.test.ann_file = root + 'test.json' # test json 정보
cfg.data.test.pipeline[1]['img_scale'] = (512,512) # Resize
#stratified k fold
cfg.data.val.classes = classes
cfg.data.val.img_prefix=root
cfg.data.val.ann_file=root+'stratified_v2/val_fold_1.json'
print(cfg.data.val.pipeline)
cfg.data.val.pipeline[1]['img_scale'] = (512,512) # Resize

cfg.data.samples_per_gpu = 4

cfg.seed = 2022
cfg.gpu_ids = [0]
#cfg.work_dir = './work_dirs/faster_rcnn_r50_fpn_1x_trash'
cfg.work_dir='./work_dirs/yolof-iter-1x'

#cfg.model.roi_heads.bbox_head.num_classes = 10
cfg.model.bbox_head.num_classes = 10


#파라미터 수정함 (학습 너무 많아서 바꿈)
cfg.optimizer_config.grad_clip = dict(max_norm=35, norm_type=2)
cfg.checkpoint_config = dict(max_keep_ckpts=3, interval=1)
cfg.device = get_device()

[{'type': 'LoadImageFromFile'}, {'type': 'MultiScaleFlipAug', 'img_scale': (512, 512), 'flip': False, 'transforms': [{'type': 'Resize', 'keep_ratio': True}, {'type': 'RandomFlip'}, {'type': 'Normalize', 'mean': [103.53, 116.28, 123.675], 'std': [1.0, 1.0, 1.0], 'to_rgb': False}, {'type': 'Pad', 'size_divisor': 32}, {'type': 'ImageToTensor', 'keys': ['img']}, {'type': 'Collect', 'keys': ['img']}]}]


In [4]:
# build_dataset
datasets = [build_dataset(cfg.data.train)]
#valid까지 확인
#datasets=[build_dataset(cfg.data.train),build_dataset(cfg.data.val)]

loading annotations into memory...
Done (t=0.19s)
creating index...
index created!


In [5]:
# dataset 확인
datasets[0]



CocoDataset Train dataset with number of images 3914, and instance counts: 
+-------------------+-------+---------------+-------+-----------------+-------+-------------+-------+--------------+-------+
| category          | count | category      | count | category        | count | category    | count | category     | count |
+-------------------+-------+---------------+-------+-----------------+-------+-------------+-------+--------------+-------+
| 0 [General trash] | 3161  | 1 [Paper]     | 5115  | 2 [Paper pack]  | 706   | 3 [Metal]   | 769   | 4 [Glass]    | 835   |
| 5 [Plastic]       | 2350  | 6 [Styrofoam] | 1026  | 7 [Plastic bag] | 4151  | 8 [Battery] | 143   | 9 [Clothing] | 377   |
+-------------------+-------+---------------+-------+-----------------+-------+-------------+-------+--------------+-------+

In [6]:
# 모델 build 및 pretrained network 불러오기

model = build_detector(cfg.model)
#model.init_weights()

In [7]:
# #wandb테스트 여기서 이렇게 코드를 쳐서 실행하나 ./configs/_base_/default_runtime.py에서 수정하나 똑같은듯?
# import wandb
# wandb.init(project="level2-objectdetection", entity="hi-ai")
# cfg.log_config.hooks=[
#     dict(type='TextLoggerHook'),
#     dict(type='MMDetWandbHook',
#          init_kwargs={'project':'level2-objectdetection'},
#          interval=10,
#          log_checkpoint=True,
#          log_checkpoint_metadata=True,
#          num_eval_images=100)]

import wandb
wandb.init(project="Model", entity="hi-ai")

log_config = dict(
    interval=10,
    hooks=[
        dict(type='TextLoggerHook'),
        dict(
            type='WandbLoggerHook',
            init_kwargs=dict(project='Model', name='yolof', entity='hi-ai')
            )])
# for wandb
cfg.log_config = log_config

# import wandb
# wandb.init(project="level2-objectdetection", entity="hi-ai")


# 모델 학습

train_detector(model, datasets, cfg, distributed=False, validate=True)

# preds = []
# gts = []

# # 테스트 데이터의 예측 결과를 추출합니다.
# for i, data in enumerate(data_loader):
#     with torch.no_grad():
#         result = single_gpu_test(model, data["img"][0].cuda(), show=False)
#     preds.extend(result[0])
#     gts.extend(data["gt_labels"][0].tolist())

# # confusion matrix와 accuracy를 계산합니다.
# cm = eval_map(preds, gts, iou_thr=0.5, print_summary=False, dataset=dataset.CLASSES)
# acc = sum([1 for i, j in zip(preds, gts) if i == j]) / len(preds)

# wandb에 confusion matrix와 accuracy를 로깅합니다.
# wandb.log({"confusion_matrix": wandb.plot.confusion_matrix(preds, gts, class_names=dataset.CLASSES),
#            "accuracy": acc})

wandb: Currently logged in as: 96namsan (hi-ai). Use `wandb login --relogin` to force relogin


2023-05-11 06:37:31,347 - mmdet - INFO - Automatic scaling of learning rate (LR) has been disabled.
2023-05-11 06:37:31,443 - mmdet - INFO - Start running, host: root@718ec73291f9, work_dir: /opt/ml/baseline/mmdetection/work_dirs/yolof-iter-1x
2023-05-11 06:37:31,444 - mmdet - INFO - Hooks will be executed in the following order:
before_run:
(VERY_HIGH   ) StepLrUpdaterHook                  
(NORMAL      ) CheckpointHook                     
(LOW         ) EvalHook                           
(VERY_LOW    ) TextLoggerHook                     
(VERY_LOW    ) WandbLoggerHook                    
 -------------------- 
before_train_epoch:
(VERY_HIGH   ) StepLrUpdaterHook                  
(NORMAL      ) NumClassCheckHook                  
(LOW         ) IterTimerHook                      
(LOW         ) EvalHook                           
(VERY_LOW    ) TextLoggerHook                     
(VERY_LOW    ) WandbLoggerHook                    
 -------------------- 
before_train_iter:
(VERY_HIGH

loading annotations into memory...
Done (t=0.02s)
creating index...
index created!


2023-05-11 06:37:43,185 - mmdet - INFO - Saving checkpoint at 1 iterations
2023-05-11 06:37:44,481 - mmdet - INFO - Saving checkpoint at 2 iterations
2023-05-11 06:37:45,642 - mmdet - INFO - Saving checkpoint at 3 iterations
2023-05-11 06:37:46,884 - mmdet - INFO - Saving checkpoint at 4 iterations
2023-05-11 06:37:47,982 - mmdet - INFO - Saving checkpoint at 5 iterations
2023-05-11 06:37:49,155 - mmdet - INFO - Saving checkpoint at 6 iterations
2023-05-11 06:37:50,363 - mmdet - INFO - Saving checkpoint at 7 iterations
2023-05-11 06:37:51,686 - mmdet - INFO - Saving checkpoint at 8 iterations
2023-05-11 06:37:53,131 - mmdet - INFO - Saving checkpoint at 9 iterations
2023-05-11 06:37:54,517 - mmdet - INFO - Saving checkpoint at 10 iterations
2023-05-11 06:37:55,923 - mmdet - INFO - Iter [10/15000]	lr: 7.995e-04, eta: 5:46:07, time: 1.385, data_time: 0.025, memory: 1461, loss_cls: 4.3771, loss_bbox: 0.8545, loss: 5.2316, grad_norm: 90.8023
2023-05-11 06:37:56,192 - mmdet - INFO - Saving 

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 969/969, 21.0 task/s, elapsed: 46s, ETA:     0s

...

2023-05-11 08:55:24,790 - mmdet - INFO - 
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.014
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=1000 ] = 0.037
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=1000 ] = 0.008
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=1000 ] = 0.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=1000 ] = 0.002
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=1000 ] = 0.017
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.125
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=300 ] = 0.125
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=1000 ] = 0.125
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=1000 ] = 0.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets=1000 ] = 0.001
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= large | maxDets=1000

DONE (t=2.97s).


2023-05-11 08:55:25,131 - mmdet - INFO - Saving checkpoint at 6001 iterations
2023-05-11 08:55:26,468 - mmdet - INFO - Saving checkpoint at 6002 iterations
2023-05-11 08:55:27,914 - mmdet - INFO - Saving checkpoint at 6003 iterations
2023-05-11 08:55:29,376 - mmdet - INFO - Saving checkpoint at 6004 iterations
2023-05-11 08:55:31,008 - mmdet - INFO - Saving checkpoint at 6005 iterations
2023-05-11 08:55:32,519 - mmdet - INFO - Saving checkpoint at 6006 iterations
2023-05-11 08:55:33,931 - mmdet - INFO - Saving checkpoint at 6007 iterations
2023-05-11 08:55:35,492 - mmdet - INFO - Saving checkpoint at 6008 iterations
2023-05-11 08:55:36,916 - mmdet - INFO - Saving checkpoint at 6009 iterations
2023-05-11 08:55:38,349 - mmdet - INFO - Saving checkpoint at 6010 iterations
2023-05-11 08:55:39,701 - mmdet - INFO - Iter [6010/15000]	lr: 1.200e-02, eta: 3:26:21, time: 5.768, data_time: 4.305, memory: 1461, loss_cls: 0.5442, loss_bbox: 0.6690, loss: 1.2133, grad_norm: 0.5953
2023-05-11 08:55:3

OSError: [Errno 28] No space left on device

--- Logging error ---
Traceback (most recent call last):
  File "/opt/conda/envs/detection/lib/python3.7/logging/__init__.py", line 1029, in emit
    self.flush()
  File "/opt/conda/envs/detection/lib/python3.7/logging/__init__.py", line 1009, in flush
    self.stream.flush()
OSError: [Errno 28] No space left on device
Call stack:
  File "/opt/conda/envs/detection/lib/python3.7/threading.py", line 890, in _bootstrap
    self._bootstrap_inner()
  File "/opt/conda/envs/detection/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/opt/conda/envs/detection/lib/python3.7/site-packages/wandb/sdk/internal/internal_util.py", line 49, in run
    self._run()
  File "/opt/conda/envs/detection/lib/python3.7/site-packages/wandb/sdk/internal/internal_util.py", line 101, in _run
    self._finish()
  File "/opt/conda/envs/detection/lib/python3.7/site-packages/wandb/sdk/internal/internal.py", line 282, in _finish
    self._hm.finish()
  File "/opt/conda/envs/detection/lib